<a href="https://colab.research.google.com/github/hyesukim1/Face-Verification-Project/blob/main/mtcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mctnn 모델


In [ ]:
class MTCNN(nn.Module):
    """MTCNN face detection module.

    This class loads pretrained P-, R-, and O-nets and, given raw input images as PIL images,
    returns images cropped to include the face only. Cropped faces can optionally be saved also.
    
    Keyword Arguments:
        image_size {int} -- Output image size in pixels. The image will be square. (default: {160})
        margin {int} -- Margin to add to bounding box, in terms of pixels in the final image. 
            Note that the application of the margin differs slightly from the davidsandberg/facenet
            repo, which applies the margin to the original image before resizing, making the margin
            dependent on the original image size. (default: {0})
        min_face_size {int} -- Minimum face size to search for. (default: {20})
        thresholds {list} -- MTCNN face detection thresholds (default: {[0.6, 0.7, 0.7]})
        factor {float} -- Factor used to create a scaling pyramid of face sizes. (default: {0.709})
        prewhiten {bool} -- Whether or not to prewhiten images before returning. (default: {True})
        select_largest {bool} -- If True, if multiple faces are detected, the largest is returned.
            If False, the face with the highest detect probability is returned. (default: {True})
        keep_all {bool} -- If True, all detected faces are returned, in the order dictated by the
            select_largest parameter. If a save_path is specified, the first face is saved to that
            path and the remaining faces are saved to <save_path>1, <save_path>2 etc.
        device {torch.device} -- The device on which to run neural net passes. Image tensors and
            models are copied to this device before running forward passes. (default: {None})
    """

    def __init__(
        self, image_size=160, margin=0, min_face_size=20,
        thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
        select_largest=True, keep_all=False, device=None
    ):
        super().__init__()

        self.image_size = image_size
        self.margin = margin
        self.min_face_size = min_face_size
        self.thresholds = thresholds
        self.factor = factor
        self.prewhiten = prewhiten
        self.select_largest = select_largest
        self.keep_all = keep_all

        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()

        self.device = torch.device('cpu')
        if device is not None:
            self.device = device
            self.to(device)


    def forward(self, img, save_path=None, return_prob=False):
        """Run MTCNN face detection on a PIL image. This method performs both detection and
        extraction of faces, returning tensors representing detected faces rather than the bounding
        boxes. To access bounding boxes, see the MTCNN.detect() method below.
        
        Arguments:
            img {PIL.Image} -- A PIL image.
        
        Keyword Arguments:
            save_path {str} -- An optional save path for the cropped image. Note that when
                self.prewhiten=True, although the returned tensor is prewhitened, the saved face
                image is not, so it is a true representation of the face in the input image.
                (default: {None})
            return_prob {bool} -- Whether or not to return the detection probability.
                (default: {False})
        
        Returns:
            Union[torch.Tensor, tuple(torch.tensor, float)] -- If detected, cropped image of a face
                with dimensions 3 x image_size x image_size. Optionally, the probability that a
                face was detected. If self.keep_all is True, n detected faces are returned in an
                n x 3 x image_size x image_size tensor with an optional list of detection
                probabilities.

        Example:
        >>> from facenet_pytorch import MTCNN
        >>> mtcnn = MTCNN()
        >>> face_tensor, prob = mtcnn(img, save_path='face.png', return_prob=True)
        """

        with torch.no_grad():
            boxes, probs = self.detect(img)

            if boxes is None:
                if return_prob:
                    return None, [None] if self.keep_all else None
                else:
                    return None

            if not self.keep_all:
                boxes = boxes[[0]]

            faces = []
            for i, box in enumerate(boxes):
                face_path = save_path
                if save_path is not None and i > 0:
                    save_name, ext = os.path.splitext(save_path)
                    face_path = save_name + '_' + str(i + 1) + ext

                face = extract_face(img, box, self.image_size, self.margin, face_path)
                if self.prewhiten:
                    face = prewhiten(face)
                faces.append(face)

            if self.keep_all:
                faces = torch.stack(faces)
            else:
                faces = faces[0]
                probs = probs[0]

            if return_prob:
                return faces, probs
            else:
                return faces

    def detect(self, img):
        """Detect all faces in PIL image and return bounding boxes.

        This method is used by the forward method and is also useful for face detection tasks
        that require lower-level handling of bounding boxes (e.g., face tracking). The
        functionality of the forward function can be emulated by using this method followed by
        the extract_face() function.
        
        Arguments:
            img {PIL.Image} -- A PIL image.
        
        Returns:
            tuple(numpy.ndarray, list) -- For N detected faces, a tuple containing an
                Nx4 array of bounding boxes and a length N list of detection probabilities.
                Returned boxes will be sorted in descending order by detection probability if
                self.select_largest=False, otherwise the largest face will be returned first.

        Example:
        >>> from PIL import Image, ImageDraw
        >>> from facenet_pytorch import MTCNN, extract_face
        >>> mtcnn = MTCNN(keep_all=True)
        >>> boxes, probs = mtcnn.detect(img)
        >>> # Draw boxes and save faces
        >>> img_draw = img.copy()
        >>> draw = ImageDraw.Draw(img_draw)
        >>> for i, box in enumerate(boxes):
        ...     draw.rectangle(box.tolist())
        ...     extract_face(img, box, save_path='detected_face_{}.png'.format(i))
        >>> img_draw.save('annotated_faces.png')
        """

        with torch.no_grad():
            boxes = detect_face(
                img, self.min_face_size,
                self.pnet, self.rnet, self.onet,
                self.thresholds, self.factor,
                self.device
            )

        if len(boxes) == 0:
            return None, [None]

        if self.select_largest:
            boxes = boxes[
                np.argsort(
                    (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
                )[::-1]
            ]
        
        return boxes[:, 0:4], boxes[:, 4].tolist()

# Getting embeddings

In [ ]:
pip install facenet-pytorch

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import torch.nn as nn
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# Face recognition from images

In [ ]:
# The model is running on CPU, since it is already pre-trained and doesnt require GPU
device = torch.device('cpu') 
print('Running on device: {}'.format(device))

In [ ]:
#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device
)

In [ ]:
#Function takes 2 vectors 'a' and 'b'
#Returns the cosine similarity according to the definition of the dot product
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#cos_sim returns real numbers,where negative numbers have different interpretations.
#So we use this function to return only positive values.
def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)

# Define Inception Resnet V1 module (GoogLe Net)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define a dataset and data loader
dataset = datasets.ImageFolder('Film/Test')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

#Perfom MTCNN facial detection
#Detects the face present in the image and prints the probablity of face detected in the image.
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

# Calculate the 512 face embeddings
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).cpu()

# Print distance matrix for classes.
#The embeddings are plotted in space and cosine distace is measured.
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
for i in range(0,len(names)):
    emb=embeddings[i].unsqueeze(0)
    # The cosine similarity between the embeddings is given by 'dist'.
    dist =cos(embeddings[0],emb)  
        
dists = [[cos(e1,e2).item() for e2 in embeddings] for e1 in embeddings]
# The print statement below is
#Helpful for analysing the results and for determining the value of threshold.
print(pd.DataFrame(dists, columns=names, index=names)) 

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1,extract_face
from PIL import Image,ImageDraw
import torch
import cv2
import torch.nn as nn

#Takes 2 vectors 'a' and 'b'.
#Return the cosine similarity according to the definition of the dot product.
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#cos_sim returns real numbers,where negative numbers have different interpretations.
#So we use this function to return only positive values.
def cos(a,b):   
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)


#This is the function for doing face recognition.
def verify(embedding): 
    for i,k in enumerate(embeddings):
        for j,l in enumerate(embedding):
            #Computing Cosine distance.
            dist =cos(k,l)
               
            # Chosen threshold is 0.85. 
            #Threshold is determined after seeing the table in the previous cell.
            if dist > 0.85:
                #Name of the person identified is printed on the screen, as well as below the detecetd face (below the rectangular box).
                text=names[i]
                cv2.putText(im, text,(boxes[j][0].astype(int) ,boxes[j][3].astype(int) + 17), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,255,255), 2)
                print(text)
                
#Model running on CPU           
device = torch.device('cpu')  

#Define Inception Resnet V1 module (GoogLe Net)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cpu')

#Define MTCNN module
#Since MTCNN is a collection of neural nets and other code, 
#The device must be passed in the following way to enable copying of objects when needed internally.
#'keep_all' is kept True. All the faces present in the image will be detected.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device,keep_all=True
)

#Get cropped and prewhitened image tensor of PIL image.
img = Image.open('Film/Test/1.jpg')
img_cropped = mtcnn(img)
boxes,prob=mtcnn.detect(img)
img_draw = img.copy()
draw = ImageDraw.Draw(img_draw)

#Rectangular boxes are drawn on faces present in the image.
#The detected and cropped faces are then saved.
for i, box in enumerate(boxes):
    draw.rectangle(box.tolist())
    extract_face(img, box, save_path='Film/Test/Cropped_Face_{}.jpg'.format(i))
img_draw.save('Film/Test/Faces_Detected.jpg')

#Calculate embeddings of each cropped face and print it.
im=cv2.imread('Film/Test/Faces_Detected.jpg')
img_embedding = resnet(img_cropped)
print(img_embedding)

#print(size of img_embedding)
print(img_embedding.size())

#Call function verify. 
#Identify the person with the help of embeddings.
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
verify(img_embedding)

#'Image' window opens.
#The PIL image now have rectangular boxes on detected faces.
#The identified faces have their respective name below the box.
cv2.imshow("Image",im)
k=cv2.waitKey(0)

#13 is for 'Enter' key.
#If 'Enter' key is pressed, all the windows are made to close forcefully.
if k==13:
    cv2.destroyAllWindows()    